In [1068]:
import numpy as np

In [1069]:
folder = 'data_p'#'data/Thresh0.25/data_N4C0.01Th0.25'
outfile = 'simulation_params_N4C0.01Th0.25.dat'#'params/N4/Th0.25/simulation_params_N4C0.01Th0.25.dat'
maxN = 3001

In [1070]:
# No. of agents
N = 4
# Threshold
Th = 0.25

In [1071]:
Amx = np.floor(N*Th)

In [1072]:
#Read earnings sequences
earn_data = []
for n in range(0,maxN,5):
    earnings = []
    with open(folder+"/earnings_evol_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = list(map(int,line.split()))
            earnings.append(v[1:])
    earn_data.append(earnings)
earn_data = np.array(earn_data)
earn_data.shape

(601, 3001, 4)

### Efficiency
Average utility per agent over the last $\tau$ rounds of the $m$th simulation.
$$E_m=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\frac{1}{N}\sum_{i=1}^NU_{i,m}(t)=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\bar{U}_{m}(t),$$
where $N$ is the number of agents, and $T$ is the length of a simulation measured in rounds. $U_{i,m}(t)$ is the utility of agent $i$ in simulation $m$ at round $t$, scaled each round as
$$U_i=\frac{u_i}{N\cdot|u|_{max}},$$
where $u_i$ is the plain utility of agent $i$, $|\cdot|_{max}$ is the maximum absolute $u_i$ value for the round in a particular simulation. With this choice of normalization the utility of a single agent is bounded by $-1/N\leq U_i\leq 1/N$, and the total utility per round is bounded between -1 and 1. The downside is that it is not possible to compare for the net utilities of different simulations. If this is needed, it is better only to normalize by $N$.

### Inequality
Average standard deviation of the utility per agent over the last $\tau$ rounds of the $m$th simulation.
$$Ieq_m=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\sqrt{\frac{1}{N-1}\sum_{i=1}^N(U_{i,m}(t)-\bar{U}_m(t))^2}=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\sigma_m(t).$$

In [1073]:
reset_time = 2000
tau = 2000 #earn_data.shape[1]-1000

In [1074]:
# Slice the last tau steps
earn_data = earn_data[:,-tau:,:]
earn_data.shape

(601, 2000, 4)

In [1075]:
def unorm(row):
    mx = np.abs(row).max()
    N = len(row)
    if mx != 0:
        return np.array([e/(N*mx) for e in row])
    else:
        return row

def nnorm(row):
    N = len(row)
    return np.array([e/N for e in row])

In [1076]:
# Scale utility by N
utilities = np.array([[e/(reset_time+i) for i, e in enumerate(earnings)] for earnings in earn_data])
#utilities = np.array([[nnorm(e) for e in earnings] for earnings in earn_data])
utilities.shape

(601, 2000, 4)

In [1077]:
# Scale utility full
n_utilities = np.array([[unorm(e) for e in earnings] for earnings in earn_data])
n_utilities.shape

(601, 2000, 4)

In [1078]:
N = utilities.shape[2]

In [1079]:
#Efficiencies
efficiencies = np.array([np.array([u.mean() for u in util]).mean() for util in utilities])
efficiencies.shape

(601,)

In [1080]:
#Inequalities
ineq_arrays = [np.array([u.std() for u in util]) for util in utilities]
inequalities = [np.array(list_std).mean() for list_std in ineq_arrays]
#inequalities

In [1081]:
def slope(arr):
    return (arr[-1]-arr[1])/(len(arr)-1)

av_ineq_slopes = [slope(list_std) for list_std in ineq_arrays]

In [1082]:
n_efficiencies = [np.array([u.mean() for u in util]).mean() for util in n_utilities]

In [1083]:
n_inequalities = [np.array([u.std() for u in util]).mean() for util in n_utilities]

### Entropy per agent
The entropy per agent of the sequence of length $\tau$ of states of $N$ bits (agents) during simulation $m$ is
$$h=-\frac{1}{N}\sum_kp_{k}\log_2p_{k},$$
where $p_{k}$ is the probability of finding state $k$ $(1\leq k\leq 2^N)$ in the sequence of length $\tau$ at simulation $m$
$$p_{k}=\frac{n_{k}}{\sum_{k'}n_{k'}}=\frac{n_{k}}{\tau},$$
where $n_{k}$ is the number of occurences of state $k$ in the sequence of simulation $m$. 

[//]: # "The maximum entropy per agent for a system with $N$ agents is $\log_2(2^N)/N=1$."

In [1084]:
def bin_to_dec(b_str):
    if len(b_str) > 1:
        return int(b_str[0])*2**(len(b_str)-1) + bin_to_dec(b_str[1:])
    else:
        return int(b_str[0])

In [1085]:
def dec_to_bin(dec):
    if dec > 0:
        return dec_to_bin(dec//2) + str(dec%2)
    else:
        return ""

In [1086]:
#Read state sequences
state_bins = []
state_data = []
for n in range(0,maxN,5):
    bins = []
    states = []
    with open(folder+"/state_evol_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = line.split()
            bins.append(v[1])
            states.append(bin_to_dec(v[1]))
    state_bins.append(bins)
    state_data.append(states)
state_bins = np.array(state_bins)
state_data = np.array(state_data)
state_data.shape

(601, 3001)

In [1087]:
#Slice the last tau steps
state_data = state_data[:,-tau:]
state_data.shape

(601, 2000)

In [1088]:
#Slice the last tau steps
state_bins = state_bins[:,-tau:]
state_bins.shape

(601, 2000)

In [1089]:
#Compute the distributions
def comp_dist(sequence, N):
    #dist = np.zeros(2**N)
    dist = {}
    tot = 0
    for seq in sequence:
        dist[seq] = dist.get(seq, 0) + 1
        tot += 1
    for i in dist:
        dist[i] /= tot
    return dist

#Compute the entropy per agent
def entropy(distribution, N):
    h = 0
    for i in distribution:
        p = distribution[i]
        h -= p * np.log2(p)
    return h/N

In [1090]:
#Entropies
entropies = [entropy(comp_dist(states, N), N) for states in state_data]
#entropies

### Conditional entropy (normalized)
The joint probability of state $X$ being $k$ at $t$ and $k$ at $t+1$ is
$$p(X_{t+1}=k',X_t=k)=\frac{n_{k,k'}}{\sum_{l,l'}n_{l,l'}},$$
while the conditional probability is
$$p(X_{t+1}=k'|X_t=k)=\frac{p(X_{t+1}=k',X_t=k)}{p(X_t=k)}=\frac{n_{k,k'}}{\sum_{l,l'}n_{l,l'}}\frac{\sum_ln_l}{n_k}=\frac{\tau}{\tau-1}\frac{n_{k,k'}}{n_k}.$$
The conditional entropy is given by
$$h_c=-\frac{1}{N}\sum_{k,k'}p(X_{t+1}=k',X_t=k)\log_2p(X_{t+1}=k'|X_t=k)=-\frac{1}{N}\sum_{k,k'}\frac{n_{k,k'}}{\tau-1}\log_2\left(\frac{\tau}{\tau-1}\frac{n_{k,k'}}{n_k}\right),$$
where $n_{k,k'}$ is the number of transitions from $k$ to $k'$, $n_k$ the number of occurrances of state $k$, and $\tau$ is the length of the simulation.

In [1091]:
seq_pairs = []
for n in range(state_bins.shape[0]):
    pairs = []
    for i in range(state_bins.shape[1]-1):
        pair = state_bins[n,i]+state_bins[n,i+1]
        pairs.append(bin_to_dec(pair))
    seq_pairs.append(pairs)
seq_pairs = np.array(seq_pairs)
seq_pairs.shape

(601, 1999)

In [1092]:
def comp_trans_dist(pair_sequence, N):
    pair_dist = {}
    tot = 0
    for seq in pair_sequence:
        pair_dist[seq] = pair_dist.get(seq, 0) + 1
        tot += 1
    for i in pair_dist:
        pair_dist[i] /= tot
    return pair_dist

def cond_entropy(distribution, pair_distribution, N):
    Hc = 0
    for i in pair_distribution:
        pbin = dec_to_bin(i)
        bk = pbin.zfill(2*N)[:N] #first N binary digits
        k = bin_to_dec(bk)
        p = distribution[k]
        pp = pair_distribution[i]
        if p > 0 and pp > 0:
            Hc -= pp * np.log2(pp/p)
    return Hc/N

In [1093]:
#dist = comp_dist(state_data[0], N)
#pair_dist = comp_trans_dist(seq_pairs[0], N)

In [1094]:
#for i in range(len(dist)):
#    if dist[i] > 0:
#        print(dec_to_bin(i).zfill(N), dist[i])

In [1095]:
#for i in range(len(pair_dist)):
#    if pair_dist[i] > 0:
#        print(dec_to_bin(i).zfill(2*N), pair_dist[i])

In [1096]:
#cond_entropy(dist, pair_dist)

In [1097]:
#comp_dist(state_data[0], N)

In [1098]:
#comp_trans_dist(seq_pairs[0], N)

In [1099]:
#Conditional entropies
cond_entropies = np.array([cond_entropy(comp_dist(states, N), comp_trans_dist(pair_st, N), N) for states, pair_st in zip(state_data,seq_pairs)])
cond_entropies = np.clip(cond_entropies,0,100)

In [1100]:
cond_entropies.max()

np.float64(0.09885314692656247)

### Frequency
Frequency with wich a state pattern repeats in the time series. The frequency is computed as the inverse of the period thet the pattern presents. This parameter is not well defined when the system does not reach a steady pattern, which can happen when the agents have an exploration parameter that might supersede the expected payoff value.

In [1101]:
##The fft is not getting the correct frequencies in several cases, in particular is  period>3
#def get_highest_freq(series):
#    X = fft(series)
#    sr = 1     #sampling rate (1 state every time step)
#    N = len(X)
#    n = np.arange(N)
#    T = N/sr
#   freq = n/T
#   return freq[list(np.abs(X[1:])).index(max(np.abs(X[1:int(N/2)])))]

def get_period(series, j):
    if j >= len(series): print(j) ######
    test = True
    count = 1
    while test:
        val = series[j]
        #print(val)
        prev_val = val
        count = 1
        for i in range(j+1,len(series)):
            if (series[i] == val) or (series[i] == prev_val) or (j+1==len(series)):
                test = False
                break
            #print(series[i])
            count += 1
            prev_val = series[i]
        j += 1
    return count

In [1102]:
#Frequencies
frequencies = [1/get_period(states,1) for states in state_data]
#frequencies

In [1103]:
#for i in range(len(frequencies)):
#    if frequencies[i] == 0:
#       print(state_data[i])

### Information per agent
Every agent develops a strategy to choose its next action given its knowledge of the previous state. This strategy is stored in its transition matrix and utility function. However, some strategies might be simple and depend on little to no knowledge as, for example, the strategy to follow always the same action irrespective of the previous state. Other strategies might be more complex, and depend on the knowledge of a finite number of bits from the previous state.</b>

The information per agent of a particular simulation refers to the average number of bits per agent necessary to follow the strategy the agents have adopted. As in some cases, as when using exploration, the system does not acheive a steady state and there is no fixed pattern that allows us to deduce the specific strategies of the agents. because this pattern might change in time it is necessary to do a sampling of the data. Each sample consists in randomly choosing a point in the series and obtaining the pattern that follows, by counting states until we find the initial state again (a cycle). With the obtained pattern we can then extract the strategies of the agents and compute the average information needed to sustain the pattern. This proces is repeated several times and an average is obtained.

In [1104]:
neighbors = []
for n in range(0,maxN,5):
    neigh = []
    with open(folder+"/known_idx_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = list(map(int,line.split()))
            neigh.append(v[1:])
    neighbors.append(neigh)
neighbors = np.array(neighbors)
neighbors.shape

(601, 4, 4)

In [1105]:
def get_pattern(data, j):
    p = get_period(data, j)
    mx = min(j+p,len(data))
    pat = data[j:mx]
    pattern = {}
    for i in range(len(pat[0])):
        pattern[i] = [pat[j][i] for j in range(len(pat))]
    return pattern

def filter_pattern(pattern, f):
    f_pat = {}
    for i in pattern.keys():
        if f[i]:
            f_pat[i] = pattern[i]
    return f_pat

def combinations(n, k):
    combos = []
    if (k == 0):
        return [[]]
    elif (k == 1):
        return [[i] for i in n] 
    for i in range(len(n)): 
        head = n[i:i+1]

        tail = combinations(n[i+1:],k-1)

        for j in range(len(tail)):
            #print("tail[j]", tail[j])
            if (type(tail[j]) == int):
                combo = head + [tail[j]]
            else:
                combo = head + tail[j]
            combos.append(combo)   
    return combos

def get_strategy(pattern, idx, known_idx):
    my_neighb = np.delete(known_idx[idx],np.where(known_idx[idx]==idx)[0]).tolist()
    N = len(pattern)
    l = len(pattern[idx])
    #print(my_neighb)

    for Nneigh in range(0,len(my_neighb)+1):
        neighb_lists = combinations(my_neighb, Nneigh)
        neighb_lists = [[idx] + nl for nl in neighb_lists]
        #print(neighb_lists)
        
        for n in neighb_lists:
            columns = sorted(n)
            mask = [(i in columns) for i in range(N)]
            f_pat = filter_pattern(pattern, mask)
            
            #print(f_pat)
            strat = {}
            test = True
            for i in range(l):
                p = ''.join(f_pat[a][i] for a in columns)
                #print(p)
                if p not in strat:
                    strat[p] = f_pat[idx][(i+1)%l]
                else:
                    if strat[p] != f_pat[idx][(i+1)%l]:
                        test = False
                        break
                test = True
        
            if test:   
                return tuple(columns), strat
    return None  #If the output is None, then most likely there is no stable pattern


In [1106]:
# for NNN in range(state_data.shape[0]):
#    pat = get_pattern(state_bins[NNN], 1)
#    print(pat)
#    print(neighbors[NNN])
#    strat = get_strategy(pat, 0, neighbors[NNN])
#    min_bits = 1000
#    if strat:
#        print(strat)
#        min_bits = len(strat[0]) if len(strat[1]) > 1 else 0
#    else:
#        print("Most likely there is no stable pattern")
#    print(min_bits)
#    print()

In [1107]:
#NNN = 0
#ini_t = 20
#pat = get_pattern(state_bins[NNN], ini_t)
#print(pat)
#print(neighbors[NNN])
#for i in range(N):
#    strat = get_strategy(pat, i, neighbors[NNN])
#    min_bits = 1000
#    if strat:
#        print(strat)
#        min_bits = len(strat[0]) if len(strat[1]) > 1 else 0
#    else:
#        print("Most likely there is no stable pattern")
#    print(min_bits)
#print()

In [1108]:
#ar = np.array([[0 for i in range(2**N)] for j in range(2**N)])
#ar[]

In [1109]:
def av_min_info(state, neighbors, N, M):
    mx = len(state)/3 #int(2**N)
    tot_av = 0
    for m in range(M):
        av = 0
        idx = np.random.randint(1,len(state)-mx)
        pattern = get_pattern(state, idx)
        for i in pattern.keys():
            strat = get_strategy(pattern,i,neighbors)
            if strat:
                av += len(strat[0]) if len(strat[1]) > 1 else 0
            else:
                #print(pattern)
                av += 1000
        av /= len(pattern)
        tot_av += av
    return tot_av/M

In [1110]:
#Information per agent
M = 10 #No. of samples
info_per_agent = [av_min_info(state_bins[i],neighbors[i],N,M) for i in range(len(state_bins))]
#info_per_agent

In [1111]:
## Read transition matrices
#lineN = 500
#mats = []
#for i in range(N):
#    with open(folder+"/matrix_evol_{0}_{1:04d}.dat".format(i,5*NNN), "r") as fdata:
#        for line in fdata:
#            v = list(map(float,line.split()))
#            if v[0] == lineN:
#                mats.append(np.asarray(v[1:]).reshape(2**N,2**N))       
#            else:
#                pass

In [1112]:
#rounded_mats = []
#for i in range(N):
#    rounded_mats.append(np.round(mats[i],decimals=2))
#print(rounded_mats[0])

In [1113]:
#tags = {}
#for i in range(2**N):
#    bins = dec_to_bin(i).rjust(N,'0')
#    tags[i] = bins
#tags

In [1114]:
## For each agent add the rows with the same state
#contracted_rows = []
#for i in range(N):
#    cr = [0 for a in range(2**N)]
#    for j in range(2**N):
#        cr += rounded_mats[i][j,:]
#    contracted_rows.append(cr)

In [1115]:
#contracted_rows[0]

### Inter-agent entropy
In a single round of a simulation, measures the entropy of the binary states of the agents.

In [1116]:
#Compute the entropy per agent
def entropy_i(bin_state):
    N = len(bin_state[0])
    H_ac = 0
    for s in bin_state[1:]:
        p_zeros = s.count('0')/N
        p_ones = s.count('1')/N
        if p_zeros != 0 and p_ones != 0:
            H_ac += -p_zeros*np.log2(p_zeros)-p_ones*np.log2(p_ones)
    return H_ac/len(bin_state)

In [1117]:
iagent_entropy = [entropy_i(s) for s in state_bins]
#iagent_entropy

### Percentage of rounds with max efficiency 

In [1118]:
max_eff_percent = []
for ls in state_bins:
    num = 0
    for s in ls:
        if np.sum(list(map(int,list(s)))) == Amx:
            num += 1
    max_eff_percent.append(num/len(ls))

### Save parameters

In [1119]:
with open(outfile, "w") as fout:
    for i in range(state_data.shape[0]):
        fout.write("{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11}\n".format(i,efficiencies[i],inequalities[i],n_efficiencies[i],
                                                                                n_inequalities[i],entropies[i],cond_entropies[i],
                                                                                frequencies[i],info_per_agent[i],iagent_entropy[i],
                                                                                av_ineq_slopes[i],max_eff_percent[i]))

In [1120]:
#for i in range(len(efficiencies)):
#    if inequalities[i] < 1.e-3:
#        print(state_data[i])
#        print(utilities[i])

# Transfer entropy
The transfer entropy from time series $I=(i_0,i_1,\dots,i_n,\dots)$ to time reries $J=(j_0,j_1,\dots,j_n,\dots)$ is defined as [Measuring Information Transfer, T. Shreiber, 2008]
$$T_{I\rightarrow J}=\sum p(i_{n+1},i_n^{(k)},j_n^{(l)})\log\frac{p(i_{n+1}|i_n^{(k)},j_n^{(l)})}{p(i_{n+1}|i_n^{(k)})},$$
where $i_n^{(k)}=(i_n,i_{n-1},\dots,i_{n-k+1})$, and $k$, $l$ are the memory (correlation) lengths of $I$ and $J$ respectively. For a Markovian dynamics $k=l=1$. 